In [1]:
import pandas as pd
import yfinance as yf 
from sqlalchemy import create_engine
import talib as ta
import tushare as ts
ticker_engine = create_engine('sqlite:///../../../dataset/us/us_ticker_list_with_name.db')
price_engine = create_engine('sqlite:///../../../dataset/us/us_ticker_seven_year_price.db')
indicator_engine = create_engine('sqlite:///../../../dataset/us/us_ticker_with_indicator.db')

In [33]:

pro = ts.pro_api()

trade_time = pro.us_tradecal(start_date='20160101', end_date='20220701')
trade_time[trade_time.is_open]

,cal_date,is_open,pretrade_date
0,20220701,1,20220630
1,20220630,1,20220629
2,20220629,1,20220628
3,20220628,1,20220627
4,20220627,1,20220624
...,...,...,...
2369,20160105,1,20160104
2370,20160104,1,20151231
2371,20160103,0,20151230
2372,20160102,0,20151230


In [6]:
name_df = pd.read_sql('TOTAL', con=ticker_engine)
name_list = name_df.Symbol.to_list()
name_list

['A',
 'AA',
 'AAC',
 'AACG',
 'AACI',
 'AADI',
 'AAIC',
 'AAL',
 'AAMC',
 'AAME',
 'AAN',
 'AAOI',
 'AAON',
 'AAP',
 'AAPL',
 'AAQC',
 'AAT',
 'AATC',
 'AAU',
 'AAWW',
 'AB',
 'ABB',
 'ABBV',
 'ABC',
 'ABCB',
 'ABCL',
 'ABCM',
 'ABEO',
 'ABEV',
 'ABG',
 'ABGI',
 'ABIO',
 'ABM',
 'ABMD',
 'ABNB',
 'ABOS',
 'ABR',
 'ABSI',
 'ABST',
 'ABT',
 'ABTX',
 'ABUS',
 'ABVC',
 'AC',
 'ACA',
 'ACAB',
 'ACAC',
 'ACAD',
 'ACAH',
 'ACAQ',
 'ACAX',
 'ACB',
 'ACBA',
 'ACCD',
 'ACCO',
 'ACDI',
 'ACEL',
 'ACER',
 'ACET',
 'ACEV',
 'ACGL',
 'ACH',
 'ACHC',
 'ACHL',
 'ACHR',
 'ACHV',
 'ACI',
 'ACII',
 'ACIU',
 'ACIW',
 'ACKIT',
 'ACLS',
 'ACLX',
 'ACM',
 'ACMR',
 'ACN',
 'ACNB',
 'ACNT',
 'ACON',
 'ACOR',
 'ACQR',
 'ACR',
 'ACRE',
 'ACRO',
 'ACRS',
 'ACRX',
 'ACST',
 'ACT',
 'ACTG',
 'ACU',
 'ACVA',
 'ACXP',
 'ADAG',
 'ADAL',
 'ADAP',
 'ADBE',
 'ADC',
 'ADCT',
 'ADER',
 'ADES',
 'ADEX',
 'ADGI',
 'ADI',
 'ADIL',
 'ADM',
 'ADMA',
 'ADMP',
 'ADN',
 'ADNT',
 'ADOC',
 'ADP',
 'ADPT',
 'ADRA',
 'ADRT',
 'ADSE',

In [25]:
for ticker in name_list:
    try:
        df = pd.read_sql('{}'.format(ticker), con=price_engine)
        df.index = pd.to_datetime(df.datetime)
        df.sort_index(inplace=True)
        df.to_sql('{}'.format(ticker),if_exists='replace',index=None,con=price_engine)
    except:
        continue

In [37]:
ticker = 'AAPL'
df = pd.read_sql('{}'.format(ticker), con=price_engine)
df.iloc[:df[df.datetime == '2019-04-03'].index.values[0] + 1]
df['ticker'] = ticker
if len(df) > 200:
    timeperiod = [10,20,50,150,200]
    # MA
    for period in timeperiod:
        df['MA{}'.format(period)] = ta.SMA(df.Close, timeperiod=period)
    vol_timeperiod = [20,50]
    for period in vol_timeperiod:
        df['VMA{}'.format(period)] = ta.SMA(df.Volume, timeperiod=period)
    # high&low
    df['year_high'] = df.tail(252).High.max()
    df['year_low'] = df.tail(252).Low.min()
    # rs
    df = df.tail(252)
    df['rs'] = round((df.tail(1).MA20.values[0] - df.head(1).MA20.values[0]) / df.head(1).MA20.values[0],3)
df

,Open,High,Low,Close,Volume,datetime,ticker,MA10,MA20,MA50,MA150,MA200,VMA20,VMA50,year_high,year_low,rs
1511,152.903186,153.768348,152.236921,153.440186,57808700.0,2021-09-03,AAPL,150.082999,148.534678,145.250732,132.980438,131.393007,69311065.0,77010566.0,182.186095,128.861003,0.122
1512,154.106462,156.383695,153.529693,155.816879,82278300.0,2021-09-07,AAPL,150.777110,149.061724,145.723664,133.128484,131.587974,70979545.0,77240458.0,182.186095,128.861003,0.122
1513,156.105257,156.164920,153.121974,154.245682,74420200.0,2021-09-08,AAPL,151.323051,149.534574,146.132009,133.272983,131.772067,71249400.0,77486636.0,182.186095,128.861003,0.122
1514,154.623572,155.240113,153.092145,153.211487,57305700.0,2021-09-09,AAPL,151.890872,149.942787,146.488889,133.387823,131.957422,71690010.0,77341628.0,182.186095,128.861003,0.122
1515,154.136290,154.613611,147.871393,148.139893,140893200.0,2021-09-10,AAPL,152.033075,149.946765,146.731826,133.471661,132.134691,75120540.0,78894264.0,182.186095,128.861003,0.122
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1758,161.149994,162.899994,159.820007,161.380005,73314000.0,2022-08-29,AAPL,169.061000,168.001881,154.932489,158.063765,160.519543,63811735.0,70881412.0,182.186095,128.861003,0.122
1759,162.130005,162.559998,157.720001,158.910004,77906200.0,2022-08-30,AAPL,167.649001,167.957980,155.483139,158.062354,160.577790,64711695.0,69749130.0,182.186095,128.861003,0.122
1760,160.309998,160.580002,157.139999,157.220001,87991100.0,2022-08-31,AAPL,165.916000,167.524001,155.913908,158.050275,160.617030,64985875.0,69888942.0,182.186095,128.861003,0.122
1761,156.639999,158.419998,154.669998,157.960007,74229900.0,2022-09-01,AAPL,164.297002,167.143002,156.369863,158.046249,160.659921,65923665.0,69905356.0,182.186095,128.861003,0.122


In [8]:
def get_daily_indicator(target_date):
    name_df = pd.read_sql('TOTAL', con=ticker_engine)
    ticker_list = name_df.Symbol.to_list()
    total_df = pd.DataFrame()
    for ticker in ticker_list:
        try:
            df = pd.read_sql('{}'.format(ticker), con=price_engine)
            df.dropna(inplace=True)
            df = df.iloc[:df[df.datetime == '{}'.format(target_date)].index.values[0] + 1]
            if len(df) > 200:
                df['ticker'] = ticker
                timeperiod = [10,20,50,150,200]
                # MA
                for period in timeperiod:
                    df['MA{}'.format(period)] = ta.SMA(df.Close, timeperiod=period)
                vol_timeperiod = [20,50]
                for period in vol_timeperiod:
                    df['VMA{}'.format(period)] = ta.SMA(df.Volume, timeperiod=period)
                # high&low
                df['year_high'] = df.tail(252).High.max()
                df['year_low'] = df.tail(252).Low.min()
                # rs
                df = df.tail(252)
                df['rs'] = round((df.tail(1).MA20.values[0] - df.head(1).MA20.values[0]) / df.head(1).MA20.values[0],3)
                df = df.tail(1)
                total_df = pd.concat([total_df, df],ignore_index=True)
            else:
                continue
        except:
            continue
    total_df.to_sql('{}'.format(target_date),if_exists='replace',index=None,con=indicator_engine)

In [9]:
# from script_get_fundamentals_us import get_pe
# from script_get_fundamentals_us import get_npm
# from script_get_ticker_indicator import get_ticker_indicator_his as gth
# multiprocessing
import multiprocessing


if __name__ == '__main__':

    p_1 = multiprocessing.Process(target=get_daily_indicator, args=('2019-10-01',)) 
    p_2 = multiprocessing.Process(target=get_daily_indicator, args=('2019-07-01',))
    p_3 = multiprocessing.Process(target=get_daily_indicator, args=('2020-04-01',))
    p_4 = multiprocessing.Process(target=get_daily_indicator, args=('2020-07-01',))
    p_5 = multiprocessing.Process(target=get_daily_indicator, args=('2020-010-01',))

    # p_roe.start()
    # p_pe.start()
    p_1.start()
    p_2.start()
    p_3.start()
    p_4.start()
    p_5.start()

    p_1.join()
    p_2.join()
    p_3.join()
    p_4.join()
    p_5.join()

    # p_roe.join()
    # p_pe.join()
    # get_rev(300)

    # p_roe.join()
    # p_pe.join()
    # get_npm(6043)
    # gth('2017-01-01', '2019-04-02')

    print('Done')

# get_roe(2553) # last run: 2022/8/25 22:32
# get_pe(325)

Done


In [21]:
ticker

'AAPL'

In [23]:
df = df.sort_index()
df.to_sql('{}'.format(ticker),if_exists='replace',index=None,con=price_engine)

1763

In [12]:
200/7*5

142.85714285714286

In [ ]:
for ticker in name_list:
    try:
        # print('{} start'.format(ticker))
        df = data['{}'.format(ticker)]
        df.dropna(inplace=True)
        df['symbol'] = ticker
        df['datetime'] = df.index
        timeperiod = [10,20,50,150,200]
        # MA
        for period in timeperiod:
            df['MA{}'.format(period)] = ta.SMA(df.Close, timeperiod=period)
        vol_timeperiod = [20,50]
        for period in vol_timeperiod:
            df['VMA{}'.format(period)] = ta.SMA(df.Volume, timeperiod=period)
        # high&low
        df['year_high'] = df.tail(252).High.max()
        df['year_low'] = df.tail(252).Low.min()
        # rs
        df = df.tail(252)
        df['rs'] = round((df.tail(1).MA20.values[0] - df.head(1).MA20.values[0]) / df.head(1).MA20.values[0],3)
        target = df.tail(1)
        total = pd.concat([total, target],ignore_index=True)
    except:
        print('{} error'.format(ticker))
        error_list.append(ticker)

    total.dropna(inplace=True)
    total.index = range(len(total))